In [1]:
import torch
import torch.nn as nn
from transformers import (
    RobertaTokenizer,
    Wav2Vec2FeatureExtractor,
    AutoModel,
    RobertaModel,
    HubertModel
)
import torch.optim as optim
import copy
import os
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, f1_score

try:
    from utils import get_iemocap_data_loaders, collate_fn_raw, MetricsLogger
    from models import TextOnlyModel, AudioOnlyModel
    from trainer import Trainer
    print("Successfully imported custom modules (utils, models, trainer).")
except ImportError as e:
    print(f"Error importing custom modules: {e}")
    print("Please ensure utils.py, models.py, and trainer.py exist and contain the necessary definitions.")

Successfully imported custom modules (utils, models, trainer).


In [10]:
import torch

# --- General Configuration ---
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {DEVICE}')
torch.backends.cuda.matmul.allow_tf32 = False

# Emotion labels
EMOTION_LABELS = ['angry', 'frustrated', 'happy', 'sad', 'neutral']
NUM_CLASSES = len(EMOTION_LABELS)

# Model checkpoints
AUDIO_CHECKPOINT = 'facebook/hubert-base-ls960'
TEXT_CHECKPOINT = 'roberta-base'

# Data parameters
DATA_PATH = './iemocap'
PRECOMPUTED_FEATURES = False

# Training parameters
BATCH_SIZE = 4
NUM_WORKERS = 10
LEARNING_RATE = 3e-3
N_EPOCHS = 20
PATIENCE = 5
FREEZE_BASE_MODEL = True

print("\n--- Configuration ---")
print(f"Device: {DEVICE}")
print(f"Number of classes: {NUM_CLASSES}")
print(f"Freeze base model: {FREEZE_BASE_MODEL}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Learning Rate: {LEARNING_RATE}")
print(f"Epochs: {N_EPOCHS}")
print(f"Patience: {PATIENCE}")

Using device: cuda

--- Configuration ---
Device: cuda
Number of classes: 5
Freeze base model: True
Batch Size: 4
Learning Rate: 0.003
Epochs: 20
Patience: 5


In [3]:
import torch
from transformers import RobertaTokenizer, Wav2Vec2FeatureExtractor

# --- Load Tokenizer and Processor ---
print("\nLoading tokenizer and feature extractor...")
try:
    tokenizer = RobertaTokenizer.from_pretrained(TEXT_CHECKPOINT)
    processor = Wav2Vec2FeatureExtractor.from_pretrained(AUDIO_CHECKPOINT)
    print("Tokenizer and feature extractor loaded successfully.")
except Exception as e:
    print(f"Error loading tokenizer/processor: {e}")
    raise RuntimeError("Failed to load tokenizer/processor.") from e

# --- Load Data Loaders ---
collate_wrapper = lambda b: collate_fn_raw(b, tokenizer, processor)

print("\nLoading data loaders...")
try:
    train_loader, val_loader, test_loader = get_iemocap_data_loaders(
        path=DATA_PATH,
        precomputed=PRECOMPUTED_FEATURES,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        collate_fn=collate_wrapper,
    )
    print("Data loaders created.")
    # Verify data loaders
    print("Checking a sample batch from train_loader...")
    sample_batch = next(iter(train_loader))
    print("Sample batch keys:", sample_batch.keys())
    if 'text_inputs' in sample_batch:
        print("Text input_ids shape:", sample_batch['text_inputs'].get('input_ids', torch.empty(0)).shape)
    if 'audio_inputs' in sample_batch:
        print("Audio input_values shape:", sample_batch['audio_inputs'].get('input_values', torch.empty(0)).shape)
    if 'labels' in sample_batch:
        print("Labels shape:", sample_batch['labels'].shape)

    # Store loaders for the function
    data_loaders = {
        'train': train_loader,
        'val': val_loader,
        'test': test_loader
    }

except NameError:
     print("Error: `get_iemocap_data_loaders` or `collate_fn_raw` not found. Check imports.")
     raise RuntimeError("Data loading functions not found.")
except Exception as e:
    print(f"Error creating data loaders: {e}")
    raise RuntimeError("Failed to create data loaders.") from e


Loading tokenizer and feature extractor...
Tokenizer and feature extractor loaded successfully.

Loading data loaders...
Distribution after filtering:
neutral: 1726
frustrated: 2917
angry: 1269
sad: 1250
happy: 2632
train, val, test sizes: 7835, 979, 980
Data loaders created.
Checking a sample batch from train_loader...
Sample batch keys: dict_keys(['text_inputs', 'audio_inputs', 'labels'])
Text input_ids shape: torch.Size([64, 82])
Audio input_values shape: torch.Size([64, 290400])
Labels shape: torch.Size([64])


In [4]:
import torch
import torch.optim as optim
from transformers import AutoModel
import copy
import os
import traceback

def run_unimodal_training(model_type, device, config, data_loaders):
    """
    Encapsulates the training and evaluation process for a unimodal model.

    Args:
        model_type (str): 'text' or 'audio'.
        device (str): 'cuda' or 'cpu'.
        config (dict): Dictionary containing configuration parameters like
                       checkpoints, paths, training settings.
        data_loaders (dict): Dictionary containing 'train', 'val', 'test' DataLoaders.
    """
    print(f"\n{'='*20} Starting Training for {model_type.upper()} Model {'='*20}")

    # --- Configuration Specific to this Run ---
    if model_type == 'text':
        base_checkpoint = config['text_checkpoint']
        model_save_path = config['text_model_save_path']
        metrics_save_path = config['text_metrics_save_path']
        ModelClass = TextOnlyModel
    elif model_type == 'audio':
        base_checkpoint = config['audio_checkpoint']
        model_save_path = config['audio_model_save_path']
        metrics_save_path = config['audio_metrics_save_path']
        ModelClass = AudioOnlyModel
    else:
        raise ValueError("model_type must be 'text' or 'audio'")

    num_classes = config['num_classes']
    freeze_base = config['freeze_base_model']
    lr = config['learning_rate']
    n_epochs = config['n_epochs']
    patience = config['patience']

    # --- Instantiate Logger ---
    try:
        logger = MetricsLogger(save_path=metrics_save_path)
    except NameError:
        print("Error: MetricsLogger class not found. Check imports from utils.py.")
        return # Stop this run if logger is missing

    # --- Load Base Model ---
    print(f"Loading base pre-trained model: {base_checkpoint}...")
    try:
        base_model = AutoModel.from_pretrained(base_checkpoint)
    except Exception as e:
        print(f"Error loading base model {base_checkpoint}: {e}")
        return # Stop this run

    # --- Freeze Base Model Parameters ---
    if freeze_base:
        print("Freezing base model parameters.")
        for param in base_model.parameters():
            param.requires_grad = False
    else:
        print("Base model parameters will be fine-tuned.")

    # --- Instantiate Full Model ---
    print(f"Instantiating {model_type.capitalize()}OnlyModel...")
    hidden_size = base_model.config.hidden_size
    try:
        # Instantiate the specific model class
        if model_type == 'text':
             # Pass the actual loaded base_model instance
            model = ModelClass(roberta=base_model, num_classes=num_classes, hidden_size=hidden_size)
        elif model_type == 'audio':
             # Pass the actual loaded base_model instance
            model = ModelClass(hubert=base_model, num_classes=num_classes, hidden_size=hidden_size)
        print(f"{model_type.capitalize()}OnlyModel instantiated successfully.")
    except NameError:
         print(f"Error: {ModelClass.__name__} class not found. Check imports from models.py.")
         return # Stop this run if model class is missing
    except Exception as e:
        print(f"Error instantiating {ModelClass.__name__}: {e}")
        traceback.print_exc()
        return # Stop this run

    # --- Move Model to Device ---
    try:
        model.to(device)
    except Exception as e:
        print(f"Error moving model to device {device}: {e}")
        return # Stop this run

    # --- Set Up Optimizer ---
    params_to_optimize = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = optim.AdamW(params_to_optimize, lr=lr)
    print(f"Optimizer: AdamW with LR={lr}")
    num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Number of trainable parameters: {num_trainable_params}")

    # --- Instantiate Trainer ---
    try:
        trainer = Trainer(model, optimizer, device=device)
    except NameError:
        print("Error: Trainer class not found. Check imports from trainer.py.")
        return # Stop this run

    # --- Training Loop ---
    best_val_loss = float('inf')
    best_model_state = None
    early_stopping_counter = 0

    print(f"\nStarting training for {n_epochs} epochs...")
    for epoch in range(1, n_epochs + 1):
        try:
            # Train
            train_loss, train_acc, train_f1 = trainer.train_one_epoch(data_loaders['train'], epoch)
            # Validate
            val_loss, val_acc, val_f1 = trainer.evaluate(data_loaders['val'], desc="Validate")

            print(f'\nEpoch {epoch}/{n_epochs}')
            print(f'  [Train] Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | F1: {train_f1:.4f}')
            print(f'  [Val]   Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | F1: {val_f1:.4f}')

            # Log metrics
            logger.log_train(train_loss, train_acc, train_f1)
            logger.log_val(val_loss, val_acc, val_f1)
            logger.save()

            # Check for improvement and save best model
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model_state = copy.deepcopy(trainer.model.state_dict())
                early_stopping_counter = 0
                print(f'  -> Validation loss improved to {best_val_loss:.4f}. Saving model state.')
                try:
                     torch.save({
                        'epoch': epoch,
                        'model_state_dict': best_model_state,
                        'optimizer_state_dict': optimizer.state_dict(),
                        'val_loss': best_val_loss,
                        'config': {
                             'model_type': model_type,
                             'num_classes': num_classes,
                             'hidden_size': hidden_size,
                             'base_checkpoint': base_checkpoint
                        }
                    }, model_save_path)
                     print(f"  -> Best model checkpoint saved to '{model_save_path}'")
                except Exception as e_save:
                    print(f"  -> Error saving model checkpoint: {e_save}")
            else:
                early_stopping_counter += 1
                print(f'  -> Validation loss did not improve. Counter: {early_stopping_counter}/{patience}')

            # Early stopping
            if early_stopping_counter >= patience:
                print(f'\nEarly stopping triggered after {epoch} epochs.')
                break
        except Exception as e_epoch:
            print(f"\nError during epoch {epoch}: {e_epoch}")
            traceback.print_exc()
            print("Stopping training for this model type due to error.")
            return # Stop this run

    print("\nTraining finished.")

    # --- Final Evaluation on Test Set ---
    print(f"\n--- Evaluating Best {model_type.upper()} Model on Test Set ---")
    if best_model_state is None:
         print("Warning: No best model state was saved (validation loss might not have improved). Evaluating last model state.")
         # Ensure the model used by the trainer is used for evaluation
         eval_model = trainer.model # Use the model instance from the trainer
         eval_model.eval() # Set to eval mode
    elif not os.path.exists(model_save_path):
        print(f"Warning: Best model checkpoint '{model_save_path}' not found, but best state exists in memory. Evaluating model from memory.")
        # Need to load the state into the current model instance
        try:
            model.load_state_dict(best_model_state)
            eval_model = model
            eval_model.eval()
        except Exception as e_load_mem:
             print(f"Error loading best model state from memory: {e_load_mem}. Cannot evaluate.")
             return
    else:
        print(f"Loading best model state from '{model_save_path}'...")
        try:
            checkpoint = torch.load(model_save_path, map_location=device)
            # Re-initialize architecture - important if script restarted or for consistency
            print("Re-initializing model architecture for evaluation...")
            eval_config = checkpoint.get('config', {})
            eval_num_classes = eval_config.get('num_classes', num_classes)
            eval_base_checkpoint = eval_config.get('base_checkpoint', base_checkpoint)
            eval_base_model = AutoModel.from_pretrained(eval_base_checkpoint)
            eval_hidden_size = eval_config.get('hidden_size', eval_base_model.config.hidden_size)

            if model_type == 'text':
                 eval_model = TextOnlyModel(roberta=eval_base_model, num_classes=eval_num_classes, hidden_size=eval_hidden_size)
            elif model_type == 'audio':
                 eval_model = AudioOnlyModel(hubert=eval_base_model, num_classes=eval_num_classes, hidden_size=eval_hidden_size)

            eval_model.load_state_dict(checkpoint['model_state_dict'])
            eval_model.to(device)
            eval_model.eval()
            print("Best model loaded successfully from file.")

        except FileNotFoundError:
            print(f"Error: Checkpoint file not found at {model_save_path}. Cannot evaluate.")
            return
        except Exception as e_load:
            print(f"An error occurred loading the best model: {e_load}")
            traceback.print_exc()
            return

    # Use the existing Trainer instance structure for evaluation, but with the loaded/best model
    # Create a dummy optimizer as Trainer expects one, but it won't be used for eval
    eval_optimizer = optim.AdamW(eval_model.parameters(), lr=1e-5)
    eval_trainer = Trainer(eval_model, eval_optimizer, device=device)

    print("\nRunning evaluation on the test set...")
    try:
        test_loss, test_acc, test_f1 = eval_trainer.evaluate(data_loaders['test'], desc="Test")

        print("\n--- Test Set Results ---")
        print(f"  Loss: {test_loss:.4f}")
        print(f"  Accuracy: {test_acc:.4f}")
        print(f"  Weighted F1-Score: {test_f1:.4f}")

        # Log test results
        logger.log_test(test_loss, test_acc, test_f1)
        logger.save()
        print(f"Test metrics logged and saved to {logger.save_path}")

    except Exception as e_eval:
        print(f"An error occurred during final evaluation: {e_eval}")
        traceback.print_exc()

    print(f"\n{'='*20} Finished Run for {model_type.upper()} Model {'='*20}")


print("Training and evaluation function defined.")

Training and evaluation function defined.


In [5]:
# --- Define Configuration for the Run ---
# Using variables defined in Block 2
run_config_text = {
    'text_checkpoint': TEXT_CHECKPOINT,
    'audio_checkpoint': AUDIO_CHECKPOINT, # Keep both for potential future use in config
    'text_model_save_path': 'best_text_only_model.pth',
    'text_metrics_save_path': 'text_only_training_metrics.json',
    'audio_model_save_path': 'best_audio_only_model.pth', # Keep both for consistency
    'audio_metrics_save_path': 'audio_only_training_metrics.json',
    'num_classes': NUM_CLASSES,
    'freeze_base_model': FREEZE_BASE_MODEL,
    'learning_rate': LEARNING_RATE,
    'n_epochs': N_EPOCHS,
    'patience': PATIENCE,
}

# --- Run Training for Text Model ---
try:
    # Pass necessary variables: model type, device, config dict, data loaders dict
    run_unimodal_training(
        model_type='text',
        device=DEVICE,
        config=run_config_text,
        data_loaders=data_loaders # data_loaders defined in Block 3
    )
except NameError as e:
    print(f"Error calling training function for TEXT model: Required variable not defined ({e}). Check previous blocks.")
except Exception as e:
    print(f"An unexpected error occurred during TEXT model training execution: {e}")
    traceback.print_exc()


==================== Starting Training for TEXT Model ====================
Loading base pre-trained model: roberta-base...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Freezing base model parameters.
Instantiating TextOnlyModel...
TextOnlyModel initialized (compatible version) with hidden_size=768, num_classes=5
TextOnlyModel instantiated successfully.
Optimizer: AdamW with LR=0.003
Number of trainable parameters: 1185029

Starting training for 20 epochs...


Validate: 100%|██████████| 16/16 [00:03<00:00,  4.94it/s]



Epoch 1/20
  [Train] Loss: 1.5650 | Acc: 0.2850 | F1: 0.2149
  [Val]   Loss: 1.5150 | Acc: 0.3524 | F1: 0.2558
  -> Validation loss improved to 1.5150. Saving model state.
  -> Best model checkpoint saved to 'best_text_only_model.pth'


...


Epoch 20/20
  [Train] Loss: 1.3481 | Acc: 0.4228 | F1: 0.4025
  [Val]   Loss: 1.2956 | Acc: 0.4505 | F1: 0.3932
  -> Validation loss did not improve. Counter: 1/5

Training finished.

--- Evaluating Best TEXT Model on Test Set ---
Loading best model state from 'best_text_only_model.pth'...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Re-initializing model architecture for evaluation...
TextOnlyModel initialized (compatible version) with hidden_size=768, num_classes=5
Best model loaded successfully from file.

Running evaluation on the test set...


Test: 100%|██████████| 16/16 [00:06<00:00,  2.55it/s]


--- Test Set Results ---
  Loss: 1.2648
  Accuracy: 0.4867
  Weighted F1-Score: 0.4440
Test metrics logged and saved to text_only_training_metrics.json

==================== Finished Run for TEXT Model ====================


In [11]:
# Free up memory from previous model training
import torch
import gc

# Clear CUDA cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Run garbage collection to free up memory
gc.collect()

print("Memory cleared from previous model training.")


# --- Define Configuration for the Run ---
# Re-use most config, paths are handled inside the function based on model_type
run_config_audio = {
    'text_checkpoint': TEXT_CHECKPOINT,
    'audio_checkpoint': AUDIO_CHECKPOINT,
    'text_model_save_path': 'best_text_only_model.pth',
    'text_metrics_save_path': 'text_only_training_metrics.json',
    'audio_model_save_path': 'best_audio_only_model.pth',
    'audio_metrics_save_path': 'audio_only_training_metrics.json',
    'num_classes': NUM_CLASSES,
    'freeze_base_model': FREEZE_BASE_MODEL,
    'learning_rate': LEARNING_RATE,
    'n_epochs': N_EPOCHS,
    'patience': PATIENCE,
}

# --- Run Training for Audio Model ---
try:
     # Pass necessary variables: model type, device, config dict, data loaders dict
    run_unimodal_training(
        model_type='audio',
        device=DEVICE,
        config=run_config_audio,
        data_loaders=data_loaders # data_loaders defined in Block 3
    )
except NameError as e:
    print(f"Error calling training function for AUDIO model: Required variable not defined ({e}). Check previous blocks.")
except Exception as e:
    print(f"An unexpected error occurred during AUDIO model training execution: {e}")
    traceback.print_exc()

print("\n--- All Training Runs Complete ---")

Memory cleared from previous model training.

==================== Starting Training for AUDIO Model ====================
Loading base pre-trained model: facebook/hubert-base-ls960...
Freezing base model parameters.
Instantiating AudioOnlyModel...
AudioOnlyModel initialized (compatible version) with hidden_size=768, num_classes=5
AudioOnlyModel instantiated successfully.
Optimizer: AdamW with LR=0.003
Number of trainable parameters: 1185029

Starting training for 20 epochs...


Epoch 1 [Train]:   0%|          | 0/123 [00:07<?, ?it/s]


Error during epoch 1: CUDA out of memory. Tried to allocate 3.52 GiB. GPU 0 has a total capacity of 10.00 GiB of which 0 bytes is free. Of the allocated memory 21.57 GiB is allocated by PyTorch, and 54.26 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Stopping training for this model type due to error.

--- All Training Runs Complete ---



Traceback (most recent call last):
  File "/tmp/ipykernel_13444/3535036055.py", line 114, in run_unimodal_training
    train_loss, train_acc, train_f1 = trainer.train_one_epoch(data_loaders['train'], epoch)
                                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/i/Github/csci535_project/trainer.py", line 35, in train_one_epoch
    loss, preds, labels = self.step(batch)
                          ^^^^^^^^^^^^^^^^
  File "/mnt/i/Github/csci535_project/trainer.py", line 22, in step
    logits = self.model(text_inputs, audio_inputs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/aaaab/anaconda3/envs/535ml/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1751, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/aaaab/anaconda3/envs/535ml/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1762, in _call_impl
    return forward_ca